# Part 1: Gathering Forum Data

Author: David Skarbrevik

W266 NLP course 

Fall 2017

Part 1 of Final Project

### Make a class to gather the data

In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import re
import os
import pandas as pd
from collections import defaultdict
import ast
import sys

In [30]:
tmp_data = {"user":[], "date":[], "time":[],"upvotes":[], "downvotes":[], 
                 "op?":[], "topic":[], "text":[], "user_posts":[]}

for key in tmp_data.keys():
    if tmp_data[key]:
        print("nooo")

In [73]:
for i in range(20,30):
    check = True
    for j in range(5):
        print(i,j)
    else:
        print("hi")
        check = False
        continue
#     if not check:
#         print("heello")
#         continue
    print("don't come here")

20 0
20 1
20 2
20 3
20 4
hi
21 0
21 1
21 2
21 3
21 4
hi
22 0
22 1
22 2
22 3
22 4
hi
23 0
23 1
23 2
23 3
23 4
hi
24 0
24 1
24 2
24 3
24 4
hi
25 0
25 1
25 2
25 3
25 4
hi
26 0
26 1
26 2
26 3
26 4
hi
27 0
27 1
27 2
27 3
27 4
hi
28 0
28 1
28 2
28 3
28 4
hi
29 0
29 1
29 2
29 3
29 4
hi


In [75]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import re
import os
import pandas as pd
from collections import defaultdict

class Blizzard_Forum_Scraper():
    
    overwatch_forum_url = "overwatch/22813879"
    sc2_forum_url = "sc2/40568"
    
    def __init__(self, forum_type):
        
        if forum_type not in ["sc2","overwatch"]:
            raise NameError("please set 'forum_type' paramater to 'sc2' or 'overwatch'") 
        
        self.forum_type = forum_type
        self.data_pd = pd.DataFrame()
        self.url_open_errors = []
        # this is where data will be stored until it is transfered to our Pandas Dataframe
        self.tmp_data = {"user":[], "date":[], "time":[],"upvotes":[], "downvotes":[], 
                         "op?":[], "topic":[], "text":[], "user_posts":[], "topic_id":[]}   
     
    # helper function for get_forum_data()
    def get_post_data(self, url):
        
        index = 1
        
        # loop over pages of a specific post until the post has no more pages (most will only have 1)
        while True:
            paged_url = "{0}?page={1}".format(url,index)
            for i in range(10):
                try:
                    uClient = urlopen(paged_url)
                except:
                    continue
                break
            else:
                self.url_open_errors.append(paged_url)
                continue
                
            print("Opened {}".format(paged_url))    
            page_html = uClient.read()
            uClient.close()
            page_soup = soup(page_html, "html.parser")
            
            # if no more pages, we will see an error page
            error_check = page_soup.find("section",{"class":"Error"})
            if error_check:
                print("got to the last page of a post!")
                break
                
            #if not an error page, get the data from this page  
            else:
                all_replies = page_soup.find_all("div", {"class":"TopicPost"})
                
                user_data = {"user":"", "date":"", "time":"","upvotes":"", "downvotes":"", 
                             "op?":"", "topic":"", "text":"", "user_posts":"", "topic_id":""}
                 
                topic_data = page_soup.find("section",{"class":"Topic"})["data-topic"]

                user_data["topic_id"] = re.search(r'(?<="id":)\d*', topic_data).group()

                for reply in all_replies:

                    some_data = ast.literal_eval(reply["data-topic-post"])
                    time_data = reply.find("a",{"class":"TopicPost-timestamp"})["data-tooltip-content"]
                    user_data["user"] = some_data["author"]["name"]
                    user_data["date"] = time_data.split(" ")[0]
                    user_data["time"] = time_data.split(" ")[1]
                    user_data["upvotes"] = some_data["rank"]["voteUp"]
                    user_data["downvotes"] = some_data["rank"]["voteDown"]
                    user_data["topic"] = page_soup.find("span",{"class":"Topic-title"}).text
                    user_data["text"] = reply.find("div",{"class":"TopicPost-bodyContent"}).text
                    user_data["user_posts"] = reply.find("span",{"class":"Author-posts"}).text.strip()

                    if reply["id"] == "post-1":
                        user_data["op?"] = True
                    else:
                        user_data["op?"] = False
                    
                    for key in user_data.keys():
                        self.tmp_data[key].append(user_data[key])
                index += 1

        
    def get_forum_data(self, initial_page=1, max_page=10000000, output=True):
        
        ####################################################################
        # PARAMETERS EXPLAINED                                             #                            
        # initial_page: first page to scrape.                              #                            
        # max_page: final page to scrape.                                  #                            
        # output: if True, save to file, else just keep data in Pandas DF. #                            
        ####################################################################
        
        ##########################################
        # Step 1) try to catch user input errors #
        ##########################################
        if type(initial_page) != int:
            raise TypeError("type(initial_page) was set to a '{}' but must be type 'int'".format(type(initial_page.__name__)))
        elif initial_page <= 0:
            raise Exception("initial_page must be an 'int' greater than 0")
    
        if type(max_page) != int:
            raise TypeError("type(max_page) was set to a '{}' but must be type 'int'".format(type(max_page.__name__)))
        elif max_page < initial_page:
            raise Exception("max_page must be greater than or equal to initial_page")
        if output not in (True, False):
            raise TypeError("output parameter must be a boolean.")
        
        if output:
            file_name = "{0}_database.csv".format(self.forum_type)
            save_location = os.getcwd()+"\{}".format(file_name)
            if os.path.isfile(file_name):
                raise Exception("{} already exists. Please remove that file before calling this function.".format(file_name)) 
        else:
            raise NameError('set output parameter to "csv" or "json" or False') 

        
        ##########################################################
        # Step 2) set up important variables for data collection #
        ##########################################################
        tmp_db_exists = False                                     
        for key in self.tmp_data.keys():
            if self.tmp_data[key]:
                tmp_db_exists = True
        if tmp_db_exists:
            self.tmp_data = {"user":[], "date":[], "time":[],"upvotes":[], "downvotes":[], 
                 "op?":[], "topic":[], "text":[], "user_posts":[], "topic_id":[]}   
        
        if self.forum_type == "sc2":
            forum_of_interest = self.sc2_forum_url
        elif self.forum_type == "overwatch":
            forum_of_interest = self.overwatch_forum_url

        
        ###########################################################
        # Step 3) loop over all desired pages to gather html data #
        ###########################################################
        for num_page_count,curr_page in enumerate(range(initial_page, max_page+1)):

            ##############################                                  
            # Step 3.1) download webpage #
            ############################## 
            forum_url = 'https://us.battle.net/forums/en/{specific_forum}/?page={curr_page}'.format(specific_forum = forum_of_interest,                            
                                                                                             curr_page = curr_page)
            # in case network connection issue, retry up to 10 times        
            for i in range(10):
                try:
                    uClient = urlopen(forum_url)
                except:
                    continue
                break
            else:
                self.url_open_errors.append(forum_url)
                continue
            page_html = uClient.read()
            uClient.close()
            print("{} was opened and read successfully.".format(forum_url)) # for progress checking during runtime
            page_soup = soup(page_html, "html.parser")
            
            # if user page range exceeds pages of forum this will end the loop
            no_more_pages = page_soup.find("span",{"class":"Forum-createFirstTopic"})
            if no_more_pages:
                max_page = curr_page-1
                break
            
            ###################################                                  
            # Step 3.2) get data from webpage #
            ###################################
            soup_details = page_soup.find_all("a",{"class":"ForumTopic"})
            num_post_per_page = 0
            for i in range(len(soup_details)):
                url = "https://us.battle.net"+soup_details[i]['href']
                self.get_post_data(url)

            print("There were {0} posts in page {1}.".format(len(soup_details), curr_page)) # for quality checking
            
            
            ############################################################
            # Step 3.3) save data in batches to disk or just Pandas df #
            ############################################################
            if output and (num_page_count+1 % 10) == 0:
                print("Finished reading pages {0} to {1}. Saving to disk.".format(num_page_count-8, num_page_count+1))
                if self.data_pd.empty: # first time saving so include headers
                    self.data_pd = pd.DataFrame.from_dict(self.tmp_data)
                    self.data_pd.to_csv(file_name, index=False)
                else: 
                    self.data_pd = pd.DataFrame.from_dict(self.tmp_data)
                    self.data_pd.to_csv(file_name, mode='a', header=False, index=False)
                
                self.tmp_data = {"user":[], "date":[], "time":[],"upvotes":[], "downvotes":[], 
                 "op?":[], "topic":[], "text":[], "user_posts":[], "topic_id":[]}     

        
        #########################################################################
        # Step 4) save to disk anything that wasn't already batch saved earlier #
        #########################################################################
        if output:
            if self.data_pd.empty:
                self.data_pd = pd.DataFrame.from_dict(self.tmp_data)  
                self.data_pd.to_csv(file_name, index=False)
            else:
                self.data_pd = pd.DataFrame.from_dict(self.tmp_data)
                self.data_pd.to_csv(file_name, mode='a', index=False, header=False)
               
        unopened_links = len(self.url_open_errors)
        print("Web scrape complete! Database was saved to {}".format(save_location))
        print("The last page read was {}".format(max_page))
        print("There were {} pages that could not be opened.".format(unopened_links))
        if unopened_links > 0 and unopened_links < 200:
            print("These are the pages that could not be opened:")
            for link in self.url_open_errors:
                print(link)

    
        

### Using our new scraper class

In [59]:
WebScraper = Blizzard_Forum_Scraper(forum_type="overwatch")

In [60]:
WebScraper.get_forum_data(initial_page=1, max_page=10000)

https://us.battle.net/forums/en/overwatch/22813879/?page=2 was opened and read successfully.
Opened https://us.battle.net/forums/en/overwatch/topic/20761876078?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20761876078?page=2
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20761876076?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20761876076?page=2
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20761777131?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20761777131?page=2
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20762016061?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20762016061?page=2
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20762026078?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20762026078?page=2
got to the last page of a post!
Opened 

Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=70
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=71
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=72
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=73
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=74
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=75
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=76
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=77
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=78
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=79
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=80
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=81
Opened https://us.battle.net/forums/en/overwatch/topic/20759150839?page=82
Opened https://us.battle.

Opened https://us.battle.net/forums/en/overwatch/topic/20761856049?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20761856049?page=2
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20760947874?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20760947874?page=2
Opened https://us.battle.net/forums/en/overwatch/topic/20760947874?page=3
Opened https://us.battle.net/forums/en/overwatch/topic/20760947874?page=4
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20762046066?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20762046066?page=2
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20761936068?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20761936068?page=2
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20761896048?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/207

Opened https://us.battle.net/forums/en/overwatch/topic/20761856027?page=2
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20761946051?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20761946051?page=2
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20761787437?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20761787437?page=2
Opened https://us.battle.net/forums/en/overwatch/topic/20761787437?page=3
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20761896033?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20761896033?page=2
got to the last page of a post!
Opened https://us.battle.net/forums/en/overwatch/topic/20761757292?page=1
Opened https://us.battle.net/forums/en/overwatch/topic/20761757292?page=2
Opened https://us.battle.net/forums/en/overwatch/topic/20761757292?page=3
Opened https://us.battle.net/forums/en/overwatch/topic/207

'Web scrape complete! Database was saved to C:\\Users\\skarb\\Documents\\GitHub\\Forum-User-Lifecycle-Analysis\\overwatch_database.csv'

***

## Testing Playground

In [51]:
forum_url = 'https://us.battle.net/forums/en/overwatch/22813879/?page=2'
try:
    uClient = urlopen(forum_url)
except:
    print("Error opening {}".format(forum_url))  
page_html = uClient.read()
uClient.close()
print("{} was opened and read successfully.".format(forum_url)) # for progress checking during runtime
page_soup = soup(page_html, "html.parser")

# if user page range exceeds pages of forum this will end the loop
error_check = page_soup.find("span",{"class":"Forum-createFirstTopic"})
if error_check:
    print("404 page??")
    
    
soup_details = page_soup.find_all("a",{"class":"ForumTopic"})
num_post_per_page = 0
for i in range(len(soup_details)):

    num_post_per_page += 1

    url = "https://us.battle.net"+soup_details[i]['href']
    print(url)

https://us.battle.net/forums/en/overwatch/22813879/?page=2 was opened and read successfully.
https://us.battle.net/forums/en/overwatch/topic/20761856054
https://us.battle.net/forums/en/overwatch/topic/20762016016
https://us.battle.net/forums/en/overwatch/topic/20760756896
https://us.battle.net/forums/en/overwatch/topic/20761687589
https://us.battle.net/forums/en/overwatch/topic/20761767391
https://us.battle.net/forums/en/overwatch/topic/20762016058
https://us.battle.net/forums/en/overwatch/topic/20761636637
https://us.battle.net/forums/en/overwatch/topic/20761687047
https://us.battle.net/forums/en/overwatch/topic/20761707402
https://us.battle.net/forums/en/overwatch/topic/20761886048
https://us.battle.net/forums/en/overwatch/topic/20762046057
https://us.battle.net/forums/en/overwatch/topic/20761787334
https://us.battle.net/forums/en/overwatch/topic/20761856049
https://us.battle.net/forums/en/overwatch/topic/20760947874
https://us.battle.net/forums/en/overwatch/topic/20762046066
https:/

In [17]:
tmp_data = defaultdict(lambda: [])
tmp_data2 = defaultdict(lambda: [])

tmp_data["something"].append("some value")
tmp_data2["something_else"].append("some other value")

tmp_data = {**tmp_data, **tmp_data2}
tmp_data

#                 date = soup_details[i]['data-created-date']
#                 time = soup_details[i]['data-created-time']
#                 user = soup_details[i].find('span',{"ForumTopic-author"}).text
#                 replies = soup_details[i].find('span',{"ForumTopic-replies"}).text
#                 last_reply_time = soup_details[i].find('span',{"ForumTopic-timestamp"}).text.strip()
#                 topic = soup_details[i].find('span',{"ForumTopic-title"}).text.strip()
#                 text = soup_details[i].find('span',{"ForumTopic--preview"}).text
                
#                 # make sure that we're not missing data
#                 tmp_list = [tmp_date, tmp_time, tmp_user, tmp_replies, tmp_last_reply_time, tmp_text]
#                 for val in tmp_list:
#                     if val is None:
#                         raise Exception("Received None type data")        
                
#                 self.tmp_data["user"].append(user)
#                 self.tmp_data["topic"].append(topic)
#                 self.tmp_data["time"].append(time)
#                 self.tmp_data["date"].append(date)
#                 self.tmp_data["text"].append(text)
#                 self.tmp_data["downvotes"].append(replies)
#                 self.tmp_data["upvotes"].append(last_reply_time)

{'something': ['some value'], 'something_else': ['some other value']}

In [33]:
#uClient = urlopen("https://us.battle.net/forums/en/overwatch/topic/20761647418") # test replies page
uClient = urlopen("https://us.battle.net/forums/en/overwatch/topic/20761647418?page=1") # test posts page
page_html = uClient.read()
uClient.close()
        
page_soup = soup(page_html, "html.parser")  
#soup_details = page_soup.findAll("a",{"class":"ForumTopic"})

all_replies = page_soup.find_all("div", {"class":"TopicPost"})

user_data = {"user":"", "date":"", "time":"","upvotes":"", "downvotes":"", 
             "op?":"", "topic":"", "text":"", "user_posts":"", "topic_id":""}


#user_data["topic_id"] = 
topic_data = page_soup.find("section",{"class":"Topic"})["data-topic"]

user_data["topic_id"] = re.search(r'(?<="id":)\d*', topic_data).group()

for reply in all_replies:

    some_data = ast.literal_eval(reply["data-topic-post"])
    time_data = reply.find("a",{"class":"TopicPost-timestamp"})["data-tooltip-content"]
    user_data["user"] = some_data["author"]["name"]
    user_data["date"] = time_data.split(" ")[0]
    user_data["time"] = time_data.split(" ")[1]
    user_data["upvotes"] = some_data["rank"]["voteUp"]
    user_data["downvotes"] = some_data["rank"]["voteDown"]
    user_data["topic"] = page_soup.find("span",{"class":"Topic-title"}).text
    user_data["text"] = reply.find("div",{"class":"TopicPost-bodyContent"}).text
    user_data["user_posts"] = reply.find("span",{"class":"Author-posts"}).text.strip()

    if reply["id"] == "post-1":
        user_data["op?"] = True
    else:
        user_data["op?"] = False
    

    print(user_data)

{'user': 'Terranguard', 'date': '02/12/2018', 'time': '06:58', 'upvotes': 36, 'downvotes': 24, 'op?': True, 'topic': 'make that Moira healing through barriers a feature', 'text': "I don't care if it's a bug or not It's perfect for her as a main healer. Her healing is already on a recourse and she's a main healer. She needs to be able to heal through barriers.https://www.overbuff.com/heroes go here and click competitive and you'll see Mercy is still on top.people couldn't figure out how to click the comp button where I posted this below so I had to clarify it here.", 'user_posts': '9761 posts', 'topic_id': '20761647418'}
{'user': 'alfislegend', 'date': '02/12/2018', 'time': '07:00', 'upvotes': 18, 'downvotes': 29, 'op?': False, 'topic': 'make that Moira healing through barriers a feature', 'text': "If Mei can't freeze through barriers, why the hell should Moira be able to heal through them? It's not like this bug fix will ruin her, she's the strongest healer in the game right now.", 'us

In [30]:
test = '''{ "id":20761647418, "lastPosition":90,"forum":{"id":22813879},
 "isSticky":false,"isFeatured":false,"isLocked":false,"isHidden":false,
 "isFrozen":false, "isSpam":false, "pollId":0 }'''

test = re.search(r'(?<="id":)\d*', test).group()

print(test)

20761647418


In [12]:
test = '\n\n9759 posts\n\n'.strip()
test

'9759 posts'

In [44]:
test = page_soup.find_all("a", {"class":'ForumTopic'})
links = []
print(len(test))
for i in range(len(test)):
    links.append("https://us.battle.net/"+test[i]['href'])
    print(test[i]['href'])

for link in links:
    uClient = urlopen(link)
    page_html = uClient.read()
    uClient.close()
    
    page_soup = soup(page_html, "html.parser")
    
    all_replies = page_soup.find_all("section", {"class":"TopicPost"})
    
    for reply in all_replies:
        tmp = 

48
/forums/en/overwatch/topic/20744265634
/forums/en/overwatch/topic/20744215778
/forums/en/overwatch/topic/20761088072
/forums/en/overwatch/topic/20761046611
/forums/en/overwatch/topic/20761806957
/forums/en/overwatch/topic/20761727407
/forums/en/overwatch/topic/20761627391
/forums/en/overwatch/topic/20761667418
/forums/en/overwatch/topic/20761717466
/forums/en/overwatch/topic/20761807478
/forums/en/overwatch/topic/20761827450
/forums/en/overwatch/topic/20761607422
/forums/en/overwatch/topic/20761817498
/forums/en/overwatch/topic/20761637373
/forums/en/overwatch/topic/20761727406
/forums/en/overwatch/topic/20761687591
/forums/en/overwatch/topic/20761667157
/forums/en/overwatch/topic/20761717477
/forums/en/overwatch/topic/20761777431
/forums/en/overwatch/topic/20761767349
/forums/en/overwatch/topic/20761647486
/forums/en/overwatch/topic/20761627390
/forums/en/overwatch/topic/20761687589
/forums/en/overwatch/topic/20761687614
/forums/en/overwatch/topic/20761727410
/forums/en/overwatch/t

In [69]:
import sys
try: 
    uClient = urlopen("https://us.battle.net/forums/en/overwatch/topic/20761727407/?page=2")

except:
    print("oh nooo")
print("we made it here")

    
#     page_html = uClient.read()
# uClient.close()

oh nooo
we made it here


### What we want from posts pages (for each reply/post):


* author
* num likes and dislikes [DONE]
* num posts of author
* time of posting
* text of post
* is it OP?
* text of OP


In [5]:
index = 1
while True:
    uClient = urlopen("https://us.battle.net/forums/en/overwatch/topic/20761727407?page={}".format(index))
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")
    error_check = page_soup.find("section",{"class":"Error"})
    if error_check:
        print("There's nothing on page {}!".format(index))
        break
    else:
        print("there's stuff on page {}!".format(index))

        all_replies = page_soup.find_all("div", {"class":"TopicPost"})
        likes = []
        author

        for reply in all_replies:
            likes.append(reply["data-topic-post"])
        count = 0
        for like in likes:
            tmp = ast.literal_eval(like)
            upvotes = tmp["rank"]["voteUp"]
            downvotes= tmp["rank"]["voteDown"]
            print("This post got {} score".format(upvotes - downvotes))
            count +=1
        print(count)
        index += 1

print("now we're out here!")



This post got 9 score
This post got 14 score
This post got -2 score
This post got 7 score
This post got 3 score
This post got 2 score
This post got 2 score
This post got 0 score
This post got -2 score
This post got 0 score
This post got 0 score
This post got 1 score
This post got -2 score
This post got 0 score
This post got 0 score
This post got -1 score
This post got 0 score
17
0
0
0
0
now we're out here!


In [12]:
index = 1
while True:
    uClient = urlopen("https://us.battle.net/forums/en/overwatch/topic/20761727407?page={}".format(index))
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")
    error_check = page_soup.find("section",{"class":"Error"})
    if error_check:
        print("There's nothing on page {}!".format(index))
        break
    else:
        print("there's stuff on page {}!".format(index))
    index += 1


there's stuff on page 1!
There's nothing on page 2!


In [53]:
all_replies

[<div class="TopicPost " data-topic='{ "sticky":"false","featured":"false","locked":"true","frozen":"false","hidden":"false","pollId":"0"}' data-topic-post='{"id":"207601793405","valueVoted":0,"rank":{"voteUp":21,"voteDown":12},"author":{"id":"207601283573","name":"Dancewknives"}}' id="post-1">
 <!-- Deprecated: Deeplink for existing Quotes 02/19/2016 -->
 <span id="1"></span>
 <div class="TopicPost-content">
 <aside class="TopicPost-author">
 <div class="Author-block">
 <div class="Author" data-topic-post-body-content="true" id=""><a class="Author-avatar " href="https://playoverwatch.com/en-us/career/xbl/Dancewithknives"><img alt="" src="https://blzgdapipro-a.akamaihd.net/game/unlocks/0x0250000000000C40.png"/></a><div class="Author-details"> <span class="Author-name">
 <a class="Author-name--profileLink" href="https://playoverwatch.com/en-us/career/xbl/Dancewithknives">Dancewknives</a>
 </span>
 <span class="Author-posts">
 <a class="Author-posts" data-toggle="tooltip" data-tooltip-co

In [80]:
new_user = "user5"

if new_user in data:
    print("he's there")
else:
    print("nope")
    
data[new_user] = {"text":["hiiii"],"time":["11/24/2017"]}
data_pd_again = pd.DataFrame.from_dict(data, orient="index")

he's there


In [81]:
data_pd_again

,text,time
user1,"[this, is some text they posted, some other po...","[11/2/2017, 11/3/2017]"
user2,[someone else said this thing],[11/3/2017]
user5,[hiiii],[11/24/2017]


In [ ]:
new_user = 

In [105]:
new_data_pd = pd.DataFrame.from_dict(data, orient="index")
new_data_pd

,text,time
user1,"[this, is some text they posted, some other po...","[11/2/2017, 11/3/2017]"
user2,[someone else said this thing],[11/3/2017]


In [106]:
new_data_pd.to_json("test_file.json")

In [99]:
new_data_pd

,text,time
user1,"[this, is some text they posted, some other po...","[11/2/2017, 11/3/2017]"
user2,"[someone else said this thing, some good ol text]","[11/3/2017, 11/4/2017]"


In [24]:
data_pd.head()

,text,time
user1,"[this, is some text they posted, some other po...","[11/2/2017, 11/3/2017]"
user2,someone else said this thing,11/3/2017


In [26]:
data_pd.index

Index(['user1', 'user2'], dtype='object')

In [50]:
new_data = {"user3":{"text":["making a new forum post"],"time":["11/4/2017"]}}
new_data_pd = pd.DataFrame.from_dict(new_data, orient="index")
data_pd = data_pd.append(new_data_pd)

data_pd

,text,time
user1,"[this, is some text they posted, some other po...","[11/2/2017, 11/3/2017]"
user2,[someone else said this thing],[11/3/2017]
user3,[making a new forum post],[11/4/2017]


In [45]:
data_pd

,text,time
user1,"[this, is some text they posted, some other po...","[11/2/2017, 11/3/2017]"
user2,someone else said this thing,11/3/2017
